# Sprint アンサンブル学習

In [100]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [101]:
house_price = pd.read_csv('C:/Users/miyas/kaggle/train.csv')
print(house_price.columns)
house_price.loc[:,['GrLivArea','YearBuilt','SalePrice']]

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000
5,1362,1993,143000
6,1694,2004,307000
7,2090,1973,200000
8,1774,1931,129900
9,1077,1939,118000


In [102]:
# 使用するデータのみにする
X = house_price.loc[:,['GrLivArea','YearBuilt']].values
y = house_price.SalePrice.values

# StandardScalerを使うとintをfloatに変えたっていう警告が出るので，先に変えておく．
X = X.astype(float)
y = y.astype(float)

# trainデータとtestデータに分ける
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

# 標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
sc.fit(y_train[:,np.newaxis])
y_train_std = sc.transform(y_train[:,np.newaxis]).flatten()
y_test_std = sc.transform(y_test[:,np.newaxis]).flatten()

In [103]:
print('X_train_shape:',X_train_std.shape)
print('y_train_shape:',y_train_std.shape)
print('X_test_shape:',X_test_std.shape)
print('y_test_shape:',y_test_std.shape)

X_train_shape: (1168, 2)
y_train_shape: (1168,)
X_test_shape: (292, 2)
y_test_shape: (292,)


# 【問題1】ブレンディングのスクラッチ実装
* ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を最低3つ示す．
* 精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指す

### ブレンディングとは
* N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法（＝**加重平均**）
* 最も単純には平均をとる
* 多様なモデルとは、以下のような条件を変化させることで作り出すものです。
* 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
* ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
* 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）
* 重要なのはそれぞれのモデルが大きく異なること

* 回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていない                                    
《補足》                                     
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [104]:
# 線形回帰，SVM，決定木でやってみる
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

### １．重みづけの前

In [105]:
# 線形回帰
lr = LinearRegression()
lr.fit(X_train_std,y_train_std)
y_pred_lr = lr.predict(X_test_std)
print('線形回帰MSE：',mean_squared_error(y_test_std,y_pred_lr))

# SVM
svm = SVR()
svm.fit(X_train_std,y_train_std)
y_pred_svm = svm.predict(X_test_std)
print('SVM MSE：',mean_squared_error(y_test_std,y_pred_svm))

# 決定木
tree = DecisionTreeRegressor()
tree.fit(X_train_std,y_train_std)
y_pred_tree = tree.predict(X_test_std)
print('決定木MSE：',mean_squared_error(y_test_std,y_pred_tree))

# ブレンディング
y_pred_blend = np.array([y_pred_lr,y_pred_svm,y_pred_tree])
#print('y_pred_blend_shape',y_pred_blend.shape)
y_pred_blend_mean = np.mean(y_pred_blend,axis=0)
#print('y_pred_blend_mean_shape',y_pred_blend_mean.shape)
print('ブレンディングMSE：',mean_squared_error(y_test_std,y_pred_blend_mean))

線形回帰MSE： 0.477844071097135
SVM MSE： 0.38948813290293144
決定木MSE： 0.49968449638227497
ブレンディングMSE： 0.34587545572714123


### ２．SVMの重みを大きくし，決定木の重みを小さくする
《メモ》重みづけをした場合は加重平均になる，らしい．

In [106]:
print('線形回帰MSE：',mean_squared_error(y_test_std,y_pred_lr))
print('SVM MSE：',mean_squared_error(y_test_std,y_pred_svm))
print('決定木MSE：',mean_squared_error(y_test_std,y_pred_tree))

# 決定木の結果に重み
y_pred_blend = np.array([y_pred_lr*0.2,y_pred_svm*0.7,y_pred_tree*0.1])
#print('y_pred_blend:\n',y_pred_blend)
#print('y_pred_blend_shape',y_pred_blend.shape)
y_pred_blend_mean = np.sum(y_pred_blend,axis=0)
print('ブレンディングMSE：',mean_squared_error(y_test_std,y_pred_blend_mean))

線形回帰MSE： 0.477844071097135
SVM MSE： 0.38948813290293144
決定木MSE： 0.49968449638227497
ブレンディングMSE： 0.35482720129290596


### ３．SVMのカーネルを変える

In [107]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(X_train_std,y_train_std)
svr_lin = SVR(kernel='linear', C=1e3)
svr_lin.fit(X_train_std,y_train_std)
svr_poly = SVR(kernel='poly', C=1e3, degree=3)
svr_poly.fit(X_train_std,y_train_std)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='poly', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [111]:
y_pred_rbf = svr_rbf.predict(X_test_std)
print('SVM_rbf MSE：',mean_squared_error(y_test_std,y_pred_rbf))
y_pred_lin = svr_lin.predict(X_test_std)
print('SVM_linear MSE：',mean_squared_error(y_test_std,y_pred_lin))
y_pred_poly = svr_poly.predict(X_test_std)
print('SVM_poly MSE：',mean_squared_error(y_test_std,y_pred_poly))

# linearの重みを大きく，polyの重みを小さく
y_pred_blend = np.array([y_pred_rbf*0.2,y_pred_lin*0.7,y_pred_poly*0.1])
#print('y_pred_blend:\n',y_pred_blend)
#print('y_pred_blend_shape',y_pred_blend.shape)
y_pred_blend_mean = np.sum(y_pred_blend,axis=0)
print('ブレンディングMSE：',mean_squared_error(y_test_std,y_pred_blend_mean))

SVM_rbf MSE： 1.511318529011156
SVM_linear MSE： 0.476112946512674
SVM_poly MSE： 7.448946976672513
ブレンディングMSE： 0.38254861157261427


### ４．PCAで次元を削減してブレンディングしてみる

In [118]:
"""# 対数変換
X_train_log = np.log(X_train)
X_test_log = np.log(X_test)
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)
"""


In [114]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
pca = pca.fit(X_train_std)
X_train_pca = pca.transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [125]:
# 線形回帰
lr = LinearRegression()
lr.fit(X_train_pca,y_train_std)
y_pred_lr = lr.predict(X_test_pca)
print('線形回帰MSE：',mean_squared_error(y_test_std,y_pred_lr))

# SVM
svm = SVR(kernel='rbf', C=1e3, gamma=0.1)
svm.fit(X_train_pca,y_train_std)
y_pred_svm = svm.predict(X_test_pca)
print('SVM MSE：',mean_squared_error(y_test_std,y_pred_svm))

# 決定木
tree = DecisionTreeRegressor()
tree.fit(X_train_pca,y_train_std)
y_pred_tree = tree.predict(X_test_pca)
print('決定木MSE：',mean_squared_error(y_test_std,y_pred_tree))

# ブレンディング
y_pred_blend = np.array([y_pred_lr*0.5,y_pred_svm*0.1,y_pred_tree*0.4])
y_pred_blend_mean = np.sum(y_pred_blend,axis=0)
print('ブレンディングMSE：',mean_squared_error(y_test_std,y_pred_blend_mean))

線形回帰MSE： 0.47319678982351676
SVM MSE： 2.26708340753292
決定木MSE： 0.48021830581989333
ブレンディングMSE： 0.3380313450753737


# 【問題2】バギングのスクラッチ実装
* バギングをスクラッチ実装し、単一モデルより精度があがる例を最低1つ示す
### バギングとは
* 入力データの選び方を多様化する方法
* 学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出す。
* それらによってモデルをN個学習し、推定結果の平均をとる．
* ブレンディングと異なり、それぞれの重み付けを変えることはない
* scikit-learnのtrain_test_splitを、shuffle=Trueにして使うことで、ランダムにデータを分割することができる．
* これによりブートストラップサンプルが手に入ります。
* 推定結果の平均をとる部分はブースティングと同様の実装になります。

In [126]:
# 学習する個数
N = 300
# 推定結果を入れるarray
y_pred_list = np.array([]).reshape(0,len(X_test_std))

for i in range(N):
    # trainデータとtestデータに分ける
    X_train_b,X_test_b,y_train_b,y_test_b = train_test_split(X,y,test_size=0.2,shuffle=True)
    #print('{0}回目\n{1}'.format(i,X_train_))
    # 標準化する
    sc.fit(X_train_b)
    X_train_b_std = sc.transform(X_train_b)
    X_test_b_std = sc.transform(X_test_b)
    sc.fit(y_train_b[:,np.newaxis])
    y_train_b_std = sc.transform(y_train_b[:,np.newaxis]).flatten()
    y_test_b_std = sc.transform(y_test_b[:,np.newaxis]).flatten()
    
    # 学習
    tree.fit(X_train_b_std,y_train_b_std)
    # 推定は毎回同じデータで行う（外のデータ）
    y_pred_b = tree.predict(X_test_std)
    y_pred_list = np.vstack((y_pred_list,y_pred_b))

In [127]:
#print('y_pred_shape',y_pred_list.shape)
# 全部の推定結果の平均をとる
y_pred_mean = np.mean(y_pred_list,axis=0)
#print(y_pred_mean.shape)
#print('y_pred_mean:',y_pred_mean)
mse = mean_squared_error(y_test_std,y_pred_mean)
print('バギング MSE:',mse)

# 決定木
tree = DecisionTreeRegressor()
tree.fit(X_train_std,y_train_std)
y_pred_tree = tree.predict(X_test_std)
#print(y_pred_tree.shape)
print('決定木 MSE：',mean_squared_error(y_test_std,y_pred_tree))

バギング MSE: 0.11228871192724019
決定木 MSE： 0.5133655975625967


# 【問題3】スタッキングのスクラッチ実装
* スタッキングをスクラッチ実装し、単一モデルより精度があがる例を最低1つ示す．
* 最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装する
* K=3,M=2程度する

## スタッキングの手順
### 《学習時》
#### （ステージ0）
* 学習データをK個に分割する
* 分割した内の(K−1)個をまとめて学習用データ、残り1個を推定用データとする組み合わせがK個作れる。
* あるモデルのインスタンスをK個用意し、異なる学習用データを使い学習する。
* それぞれの学習済みモデルに対して、使っていない残り1個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
* さらに、異なるモデルのインスタンスもK個用意し、同様のことを行う。
* モデルがM個あれば、M個のブレンドデータが得られる。
#### （ステージn ）
* ステージn−1のブレンドデータをMn−1次元の特徴量を持つ学習用データと考え、Kn個に分割する。以下同様
#### （ステージ N ）最後のステージ
* ステージN−1のMN−1個のブレンドデータをMN−1次元の特徴量の入力として、1種類のモデルの学習を行う。
* これが最終的な推定を行うモデルとなる。
### 《推定時》
#### （ステージ 0 ）
* テストデータをK0×M0個の学習済みモデルに入力し,K0×M0個の推定値を得る。
* これをK0の軸で平均値を求めM0次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
#### （ステージ n ）
* ステージn−1で得たブレンドテストをKn×Mn個の学習済みモデルに入力し、Kn×Mn個の推定値を得る。
* これをKnの軸で平均値を求めM0次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
#### （ステージN ）＊最後のステージ
* ステージN−1で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [128]:
# 分割数
K = 4
# KFold
from sklearn.model_selection import KFold

# ラストステージ前までの関数
def get_oof(model, X_train, y_train, X_test,K):
    kf = KFold(n_splits=K,shuffle=True)
    # trainデータのブレンドデータを入れる箱
    oof_train = np.zeros((X_train_std.shape[0]))
    
    # testデータのブレンドデータを入れる箱
    oof_test = np.zeros((X_test_std.shape[0]))
    oof_test_skf = np.array([]).reshape(0,X_test_std.shape[0])
    
    # K個のバリデーションデータを作成
    for train_index, test_index in kf.split(X_train_std):
        X_tr, X_te = X_train_std[train_index], X_train_std[test_index]
        y_tr, y_te = y_train_std[train_index], y_train_std[test_index]
        
        # それぞれのバリデーションデータで学習
        model.fit(X_tr, y_tr)
        
        # それぞれのバリデーションデータで推定（学習フェーズ）
        oof_train[test_index] = model.predict(X_te)
        #print('oof_train',oof_train[test_index])
        # 学習したmodelをtestデータで推定(推定フェーズ)
        oof_test_skf = np.vstack((oof_test_skf,model.predict(X_test_std)))
        #print('oof_test_skf',oof_test_skf)
    # testデータのpredictの平均
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [129]:
# モデルのインスタンス化
lr = LinearRegression()                 #線形回帰
svm = SVR(kernel='rbf',gamma='scale')  #SVM
tree = DecisionTreeRegressor()         #決定木

# ラストステージの手前まで学習
lr_oof_train,lr_oof_test = get_oof(lr,X_train_std,y_train_std,X_test_std,K)
sv_oof_train,sv_oof_test = get_oof(svm,X_train_std,y_train_std,X_test_std,K)
tree_oof_train,tree_oof_test = get_oof(tree,X_train_std,y_train_std,X_test_std,K)

In [130]:
# 各モデルから得たブレンドデータをくっつける
meta_train = np.concatenate((lr_oof_train,sv_oof_train,tree_oof_train),axis=1)
print('meta_train：',meta_train.shape)
print('base_：',X_train_std.shape)

# ブレンドテストの結果をくっつける
meta_test = np.concatenate((lr_oof_test,sv_oof_test,tree_oof_test),axis=1)
print('meta：',meta_test.shape)
print('base：',X_test_std.shape)

meta_train： (1168, 3)
base_： (1168, 2)
meta： (292, 3)
base： (292, 2)


In [131]:
# ラストステージはlightgbmで
import lightgbm as lgb
# LightGBM用のデータセットを作成する
train_set = lgb.Dataset(data=meta_train, label=y_train_std)
test_set = lgb.Dataset(data=meta_test,label=y_test_std)

In [132]:
params = {'task' : 'train',
          'boosting_type' : 'gbdt',
          'objective' : 'regression',
          'metric' : {'l2'},
          'num_leaves' : 31,
          'learning_rate' : 0.1,
          'feature_fraction' : 0.9,
          'bagging_fraction' : 0.8,
          'bagging_freq': 5,
          'verbose' : 1
         }
gbm = lgb.train(params,train_set,num_boost_round=100,valid_sets=test_set,early_stopping_rounds=10)

[1]	valid_0's l2: 0.982229
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.869797
[3]	valid_0's l2: 0.774438
[4]	valid_0's l2: 0.698588
[5]	valid_0's l2: 0.637235
[6]	valid_0's l2: 0.580235
[7]	valid_0's l2: 0.536522
[8]	valid_0's l2: 0.500156
[9]	valid_0's l2: 0.470461
[10]	valid_0's l2: 0.442445
[11]	valid_0's l2: 0.420988
[12]	valid_0's l2: 0.406768
[13]	valid_0's l2: 0.39171
[14]	valid_0's l2: 0.382364
[15]	valid_0's l2: 0.373055
[16]	valid_0's l2: 0.367625
[17]	valid_0's l2: 0.361514
[18]	valid_0's l2: 0.357609
[19]	valid_0's l2: 0.353598
[20]	valid_0's l2: 0.350804
[21]	valid_0's l2: 0.348641
[22]	valid_0's l2: 0.346279
[23]	valid_0's l2: 0.343986
[24]	valid_0's l2: 0.343374
[25]	valid_0's l2: 0.342125
[26]	valid_0's l2: 0.340142
[27]	valid_0's l2: 0.338939
[28]	valid_0's l2: 0.338087
[29]	valid_0's l2: 0.338188
[30]	valid_0's l2: 0.337309
[31]	valid_0's l2: 0.336664
[32]	valid_0's l2: 0.33594
[33]	valid_0's l2: 0.335801
[34]	valid_0's l2: 0.3366

In [133]:
y_pred = gbm.predict(meta_test,num_iteration=gbm.best_iteration)
print('stacking MSE：',mean_squared_error(y_test_std,y_pred))

stacking MSE： 0.33580073191621307


In [134]:
# LightGBM用のデータセットを作成する
train_single = lgb.Dataset(data=X_train_std, label=y_train_std)
test_single = lgb.Dataset(data=X_test_std,label=y_test_std)

gbm_single = lgb.train(params,train_single,num_boost_round=100,valid_sets=test_single,early_stopping_rounds=10)
y_pred_single = gbm_single.predict(X_test_std,num_iteration=gbm.best_iteration)
print('single MSE：',mean_squared_error(y_test_std,y_pred_single))

[1]	valid_0's l2: 1.0476
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.947428
[3]	valid_0's l2: 0.891787
[4]	valid_0's l2: 0.816192
[5]	valid_0's l2: 0.774592
[6]	valid_0's l2: 0.714668
[7]	valid_0's l2: 0.682924
[8]	valid_0's l2: 0.637006
[9]	valid_0's l2: 0.613492
[10]	valid_0's l2: 0.578372
[11]	valid_0's l2: 0.560209
[12]	valid_0's l2: 0.535967
[13]	valid_0's l2: 0.522324
[14]	valid_0's l2: 0.502691
[15]	valid_0's l2: 0.492492
[16]	valid_0's l2: 0.477756
[17]	valid_0's l2: 0.470162
[18]	valid_0's l2: 0.459011
[19]	valid_0's l2: 0.453265
[20]	valid_0's l2: 0.444317
[21]	valid_0's l2: 0.441111
[22]	valid_0's l2: 0.4348
[23]	valid_0's l2: 0.432545
[24]	valid_0's l2: 0.427587
[25]	valid_0's l2: 0.425999
[26]	valid_0's l2: 0.421023
[27]	valid_0's l2: 0.41943
[28]	valid_0's l2: 0.414911
[29]	valid_0's l2: 0.413886
[30]	valid_0's l2: 0.411071
[31]	valid_0's l2: 0.410563
[32]	valid_0's l2: 0.407865
[33]	valid_0's l2: 0.407587
[34]	valid_0's l2: 0.405621


# ラストステージをSVMにしてみる

In [135]:
# SVM
svm = SVR(kernel='rbf',gamma='scale')
svm.fit(X_train_std,y_train_std)
y_pred_svm = svm.predict(X_test_std)
print('SVM MSE：',mean_squared_error(y_test_std,y_pred_svm))

SVM MSE： 0.3894881329029319


In [136]:
svm_ensemble = SVR(kernel='rbf',gamma='scale')
svm_ensemble.fit(meta_train,y_train_std)
y_pred_ensemble = svm_ensemble.predict(meta_test)
print('stacking MSE:',mean_squared_error(y_test_std,y_pred_ensemble))

stacking MSE: 0.40658546211700314


# クラスにしたい．．．